<h1>Object detection script</h1>

<h2>Importing packages</h2>

In [2]:
# Import packages
from IPython.display import display, Image
from IPython import display
import csv
import os
import json

<h2>Importing a trained model from Roboflow</h2>

<h3>Create a Roboflow Project:</h3>
Sign in to your Roboflow account and create a new project, selecting "Instance Segmentation" as the task type.

<h3>Annotate your Data:</h3>
Upload your images and use the Roboflow annotation tool to label each object instance.

<h3>Splitting and Downloading the Dataset:</h3>
Once annotations are complete, split your dataset into training, validation, and test sets using Roboflow's splitting tool. Then, navigate to the "Versions" tab, select the desired version, click "Export Dataset," choose "COCO Segmentation" format, and click "Show Download Code" to generate a code snippet containing the download link.

<h3>Paste the Download Code:</h3>
Copy the entire code snippet provided by Roboflow and paste it into the designated code block below in your JupyterLab notebook.

In [3]:
# Import roboflow
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="YgqaFYQYMXIDYOCYoY2O") # Fill in your person api_key
project = rf.workspace("footage").project("seafloor_footage") # Fill in your workspace and project name
model = project.version("3").model # Fill in the number of the version

display.clear_output()
print("Successfully extracted data from roboflow")

Successfully extracted data from roboflow


<h2>Prediction</h2>

In [ ]:
def create_output_dir(base_dir):
    """
    Create a new directory if it doesn't exist.
    If the directory already exists, append a number to the directory name and create a new directory.
    
    Parameters:
    - base_dir (str): The base directory path.
    
    Returns:
    - The path of the created directory.
    """
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
        return base_dir
    else:
        counter = 2
        while True:
            new_dir = f"{base_dir}{counter}"
            if not os.path.exists(new_dir):
                os.makedirs(new_dir)
                return new_dir
            counter += 1

base_output_dir = "/cache/album/cache/kso-benjamin/bucket/kso/notebooks/test/predictions/objd-pred"
# Define the base directory where the output directories will be created.

output_dir = create_output_dir(base_output_dir)
# Create a new output directory using the base directory.

input_directory = "/cache/album/cache/kso-benjamin/bucket/kso/notebooks/test/Seafloor_footage-1/Raw/"
# Define the directory containing the raw images.

for filename in os.listdir(input_directory):
    # Iterate over the files in the input directory.
    
    filepath = os.path.join(input_directory, filename)
    # Get the full path of the current file.
    
    if filepath.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Check if the file has a valid image format (PNG, JPG, JPEG).
        
        prediction = model.predict(filepath, confidence=40, overlap=30)
        # Perform prediction on the image using the `model` object.
        
        prediction_data = prediction.json()
        # Get the JSON results of the prediction.
        
        csv_filename = os.path.splitext(filename)[0] + "_prediction.csv"
        # Generate the CSV filename based on the input filename.
        
        csv_filepath = os.path.join(output_dir, csv_filename)
        # Create the full path of the CSV file in the output directory.
        
        with open(csv_filepath, mode='w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            
            header = ["x", "y", "width", "height", "confidence", "class", "class_id", "detection_id", "image_path", "prediction_type"]
            # Define the header row of the CSV file.
            
            csv_writer.writerow(header)
            # Write the header row to the CSV file.
            
            for idx, pred in enumerate(prediction_data['predictions']):
                # Iterate over each prediction in the prediction data.
                
                row = [
                    pred.get('x', ''),
                    pred.get('y', ''),
                    pred.get('width', ''),
                    pred.get('height', ''),
                    pred.get('confidence', ''),
                    pred.get('class', ''),
                    pred.get('class_id', ''),
                    pred.get('detection_id', ''),
                    pred.get('image_path', ''),
                    pred.get('prediction_type', '')
                ]
                # Create a row of data for the current prediction.
                
                csv_writer.writerow(row)
                # Write the row to the CSV file.
        
        labeled_prediction = model.predict(filepath, confidence=40, overlap=30, labels=True)
        # Perform prediction on the image with labels using the `model` object.
        
        output_image_path = os.path.join(output_dir, os.path.splitext(filename)[0] + "_prediction.jpg")
        # Create the full path of the labeled prediction image file in the output directory.
        
        labeled_prediction.save(output_image_path)
        # Save the labeled prediction result as an image file.
        
        display.clear_output()
        # Clear the output display.
